In [ ]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"
pd.options.plotting.backend = "plotly"
from IPython.display import display, HTML, Audio

import reservoirpy
reservoirpy.verbosity(0)
from reservoirpy.nodes import Reservoir, Ridge, Input, IPReservoir

from src.algo import numpy_perlin_noise_2d

In [ ]:
data_df_list = []
for i in range(1, 31):
    data_df_list.append(pd.read_csv(f"/home/bergi/prog/python/github/parking-data/csv/2023/2023-01/2023-01-{i:02}.csv"))
data_df = pd.concat(data_df_list)
data_df["timestamp"] = pd.to_datetime(data_df["timestamp"])
data_df = data_df.set_index("timestamp").resample("1h").mean()
data_df["parken-mannheim-Hauptbahnhof-P4-Parkhaus"].plot()
#reservoir.feedback()

In [ ]:
(data_df.isna()).sum(axis=0).sort_values()

In [ ]:
data_df["parken-mannheim-Hauptbahnhof-P1-Tiefgarage"].plot()


In [ ]:
reservoir = Reservoir(100, lr=0.5, sr=0.9)
reservoir

In [ ]:
data = np.linspace(0, 1, 100).reshape(1, -1)
reservoir.reset()
states = np.concatenate([
    reservoir(data)
    for i in range(50)
])
states.shape
px.line(states[:, :10])
#print(reservoir.state())
#px.line(reservoir.state().T)
#px.line(.T)

In [ ]:
data = np.linspace(0, 6*np.pi, 100)
data = np.sin(data) + np.sin(data * 3.)
#data = np.sin(data + np.sin(data * 14)) * np.sin(data * 3.1)
X = data[:-1]
X_target = data[1:] #np.random.binomial(data > 0, .5)
px.line(pd.DataFrame({"train": X, "test": X_target}))

In [ ]:

from reservoirpy.nodes import IPReservoir, NVAR

def predict_series(input_series):
    if input_series.ndim == 1:
        input_series = input_series.reshape(1, -1, 1)
    elif input_series.ndim == 2:
        input_series = input_series.reshape(input_series.shape[0], -1, 1)
        
    #input_series = input_series.reshape(-1, 1)
    target_series = input_series[:, 1:]
    input_series = input_series[:, :-1]

    data = Input()
    res1 = Reservoir(
        1000, lr=.5, sr=.9,
        #rc_connectivity=.1,
        activation=lambda x: np.sin(x * 2.)
    )
    res2 = IPReservoir(1000, lr=.5, sr=.9, rc_connectivity=.5)
    #res2 = Reservoir(1000, lr=1., rc_connectivity=.1)
    readout1 = Ridge(ridge=1e-7) 
    #readout2 = Ridge(ridge=1e-7) 
    #res2 <<= readout1
    
    #path1 = [data >> res1, data >> res2] >> readout1
    #path2 = res1 >> res2
    #model = path1 & path2
    model = [data >> res1] >> readout1
    
    model.fit(input_series, target_series)
    #model.fit(input_series, {readout1.name: target_series, readout2.name: target_series})
    output = model.run(input_series, reset=True)
    if isinstance(output, list):
        output = np.concatenate([o[None, ...] for o in output])
    elif output.ndim == 2:
        output = output.reshape(1, -1, 1)
        
    # print(target_series.shape, output.shape)
    error_l1 = np.abs(target_series - output).mean()
    error_l2 = np.sqrt(((target_series - output) ** 2).sum())
    display(px.line(
        pd.DataFrame({
            "targets": target_series[0].reshape(-1), 
            "output": output[0].reshape(-1), 
            "error": (target_series - output)[0].reshape(-1)
        }),
        title=f"error l1 {error_l1:.3f}, l2 {error_l2:.3f}",
    ))
    
    #inp = input_series[:]
    output1 = model.run(input_series, reset=True)
    if isinstance(output1, list):
        output1 = np.concatenate([o[None, ...] for o in output1])
    elif output1.ndim == 2:
        output1 = output1.reshape(1, -1, 1)
    output1 = output1[0]
    output2 = np.zeros((30000, 1))
    x = output1[-1]
    for i in range(output2.shape[0]):
        x = model(x)
        output2[i] = x
    #print(output1.shape, output2.shape)
    display(px.line(
        np.concatenate([output1, output2])[:1000 + output1.shape[0]],
        title=f"{min(1000, output2.shape[0])} prediction after {output1.shape[0]} steps",
    ))
    display(Audio(output2.reshape(-1), rate=22050))

#data = numpy_perlin_noise_2d((10, 200), (10, 10))
data = np.linspace(0, 2.*np.pi, 1000)
data = np.sin(data) + .3 * np.sin(data * 3.)
predict_series(data)

# class prediction

In [ ]:
import torchtext.datasets
ds = torchtext.datasets.EnWik9()
wiki_texts = []
for line, _ in zip(ds, range(1000)):
    if len(line) > 300:
        wiki_texts.append(line[:3000])
display(wiki_texts[:10])
display([decode_text(encode_text(t)) for t in wiki_texts[:10]])

In [ ]:
def encode_sequence(sequence, num_classes: int):
    output = np.zeros((len(sequence), num_classes))
    for i, s in enumerate(sequence):
        output[i, s] = 1.
    return output

CHAR_MAPPING = {}
for c in " ,.:/[]0123456789":
    CHAR_MAPPING[c] = len(CHAR_MAPPING)
for c in range(ord('a'), ord('z') + 1):
    CHAR_MAPPING[chr(c)] = len(CHAR_MAPPING)
CODE_MAPPING = {
    v: k 
    for k, v in CHAR_MAPPING.items()
}
def encode_text(text: str) -> np.ndarray:
    classes = [
        CHAR_MAPPING[c]
        for c in text.lower()
        if c in CHAR_MAPPING
    ]
    return encode_sequence(classes, num_classes=len(CHAR_MAPPING))

def decode_text(code: np.ndarray) -> str:
    text = []
    for classes in code:
        c = np.argmax(classes)
        text.append(CODE_MAPPING.get(c, "?"))
    return "".join(text)
    
#encode_sequence([0, 1, 2, 3, 2, 1, 0], 4)
decode_text(encode_text("abc defz"))
#CHAR_MAPPING

In [ ]:
def predict_text(*text, max_length: int = 100, wormup: int = 0, noise: float = 0.):
    input_series = [
        encode_text(t)[None, :max_length, :]
        for t in text
    ]
    max_len = max(code.shape[1] for code in input_series)
    input_series = np.concatenate([
        np.pad(s, [(0, 0), (0, max_len - s.shape[1]), (0, 0)])
        for s in input_series
    ])
    target_series = input_series[:, 1:]
    input_series = input_series[:, :-1]
    
    data = Input()
    res1 = Reservoir(
        1000, lr=.5, sr=.9,
        rc_connectivity=.9,
        activation=lambda x: np.sin(x * 2.),
        noise_rc=noise,
    )
    #res2 = IPReservoir(1000, lr=.5, sr=.9, rc_connectivity=.5)
    #res2 = Reservoir(1000, lr=1., rc_connectivity=.1)
    readout1 = Ridge(ridge=1e-7) 
    model = [data >> res1] >> readout1
    
    model.fit(input_series, target_series, warmup=wormup)
    #model.fit(input_series, {readout1.name: target_series, readout2.name: target_series})
    output = model.run(input_series, reset=True)
    if isinstance(output, list):
        output = np.concatenate([o[None, ...] for o in output])
    elif output.ndim == 2:
        output = output.reshape(1, -1, output.shape[-1])
    
    def _plot_img(series, title):
        s = series[0].T.copy()
        s[s == 0] = np.nan
        display(px.imshow(s, title=title, aspect=False))
    
    error_l1 = np.abs(target_series - output).mean()
    error_l2 = np.sqrt(((target_series - output) ** 2).sum())

    if 1:
        _plot_img(target_series, "targets")
        _plot_img(-(target_series - output), f"output error l1 {error_l1:.3f}, l2 {error_l2:.3f}")
        display(px.line(
            pd.DataFrame({
                "targets": target_series[0, :, 0].reshape(-1), 
                "output": output[0, :, 0].reshape(-1), 
                "error": (target_series - output)[0, :, 0].reshape(-1)
            }),
            title=f"first class, {error_l1:.3f}, l2 {error_l2:.3f}",
        ))
    
    output = model.run(input_series[:1, :input_series.shape[1] // 2], reset=True)
    
    if isinstance(output, list):
        output = np.concatenate([o[None, ...] for o in output])
    elif output.ndim == 2:
        output = output.reshape(1, -1, output.shape[-1])
    
    output = output[0]
    output2 = np.zeros((1000, output.shape[-1]))
    x = output[-1]
    for i in range(output2.shape[0]):
        x = model(x)
        klass = np.argmax(x)
        x[:] = 0
        x[:, klass] = 1
        output2[i] = x
    
    output2 = np.concatenate([output, output2])
    out_text = decode_text(output2)
    print(out_text)

predict_text(
    *wiki_texts[:10],
    #"a simple text to learn to predict the next character. this is usually done with recurrent networks"
    #" which are kind of hard to train. in reservoir computing we only train the readout module"
    #" and let the reservoir rnn simply do its magic without intereferring",
    #"a second text that has nothing to do with the first except that it uses the same characters"
    #" and the same language.",
    noise=0.0,
)

In [ ]:
def act(x):
    return np.sin(x * 1.3)
    #return np.clip(x, 0, 10)

res = Reservoir(100, seed=42, sr=.9, rc_connectivity=.9, activation=act, noise_rc=.1)
#inp = np.linspace(0, 10*np.pi*2, 100).reshape(-1, 1)
inp = np.zeros((100, 1))
inp[0] = 1.
outp = res.run(inp, reset=True)
display(px.imshow(outp.T))
px.line(outp[:, :10])

In [ ]:
r = Reservoir(50, lr=0.1, sr=0.9, rc_connectivity=.5)
r(np.random.random((1, 100)))
px.imshow(r.W.toarray(), height=500)

In [ ]:
Reservoir?